In [1]:
import rasterio
import geopandas as gpd
import numpy as np
from rasterio.features import rasterize

In [2]:
# Path to the GSHHS coastline shapefile (low resolution)
coastline_shapefile = '/Users/romain/GitHub/premise-johanna/dev/gshhg-shp-2.3.7/GSHHS_shp/l/GSHHS_l_L1.shp'

# Load coastline shapefile
coastline_gdf = gpd.read_file(coastline_shapefile)


In [3]:
coastline_gdf

,id,level,source,parent_id,sibling_id,area,geometry
0,0-E,1,WVS,-1,0,5.065405e+07,"POLYGON ((180 68.99378, 180 64.99556, 179.5250..."
1,0-W,1,WVS,-1,0,5.065405e+07,"POLYGON ((-180 68.99378, -179.99844 68.99292, ..."
2,1,1,WVS,-1,1,2.922097e+07,"POLYGON ((32.27833 31.12539, 32.29581 31.21281..."
3,2,1,WVS,-1,2,2.015474e+07,"POLYGON ((-79.97678 9.36489, -79.90649 9.36414..."
4,3,1,WVS,-1,3,1.753416e+07,"POLYGON ((-73.36172 -53.00042, -73.36261 -53, ..."
...,...,...,...,...,...,...,...
5701,5829,1,WVS,-1,18438,1.367242e+00,"POLYGON ((51.25872 26.16583, 51.25875 26.16669..."
5702,5830,1,WVS,-1,18511,1.359747e+00,"POLYGON ((-17 63.78294, -16.99917 63.78247, -1..."
5703,5831,1,WVS,-1,18642,1.343374e+00,"POLYGON ((167.31797 9.10833, 167.31797 9.10914..."
5704,5832,1,WVS,-1,19324,1.264468e+00,"POLYGON ((172.79458 1.88997, 172.79586 1.89125..."


In [16]:
import numpy as np
import geopandas as gpd
from shapely.geometry import Point

def find_nearest_coastline(df, lat, lon):
    """
    Given a dataframe with coastline geometries and a lat/lon coordinate,
    returns the minimum distance to the nearest coastline in degrees.
    """
    # Convert input lat/lon into a Point
    point = Point(lon, lat)
    
    # Ensure 'geometry' column contains shapely objects
    if not isinstance(df.geometry.iloc[0], Point):
        df["geometry"] = df["geometry"].apply(lambda geom: geom if isinstance(geom, Point) else geom)

    # Compute distances (in degrees)
    df["distance"] = df["geometry"].distance(point)

    # Return the minimum distance (in degrees)
    return df["distance"].min()

def degrees_to_meters(lat, distance_deg):
    """
    Convert a distance in degrees to meters at a given latitude.

    Parameters:
    - lat (float): Latitude in degrees
    - distance_deg (float): Distance in degrees

    Returns:
    - float: Distance in meters
    """
    # Convert latitude to radians
    lat_rad = np.radians(lat)
    
    # Approximate conversion factor based on latitude
    avg_factor = (111132.92 - 559.82 * np.cos(2 * lat_rad) + 
                  1.175 * np.cos(4 * lat_rad) - 0.0023 * np.cos(6 * lat_rad))
    
    # Convert degrees to meters
    return distance_deg * avg_factor

# Example usage:
lat, lon = 45.0, -10.0  # Example query point
min_distance_deg = find_nearest_coastline(coastline_gdf, lat, lon)
min_distance_meters = degrees_to_meters(lat, min_distance_deg)

print(f"Minimum distance to the coastline: {min_distance_meters:.2f} meters")


Minimum distance to the coastline: 1.985062 degrees
Minimum distance to the coastline: 220603.46 meters


/var/folders/cn/pys1j9rn0y7djkhv3hfdtrs00000gn/T/ipykernel_8151/3793361613.py:18: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  df["distance"] = df["geometry"].distance(point)


In [4]:
from windisch import *
import xarray as xr

tip = TurbinesInputParameters()
tip.static()
_, array = fill_xarray_from_input_parameters(
    tip,
    scope={"year": [2020, ],}
)
fp = "era5_mean_2013-2022_month_by_hour_corrected.nc"
wt = WindTurbineModel(
    array,
    location=(56.443123154647964, 11.431821606429255),
    wind_data=xr.open_dataset(fp),
    country="DK"
)
wt.set_all()

Fetching terrain data only.
Using cached data for location (56.443123154647964, 11.431821606429255)
Offshore wind turbines


In [5]:
wt["distance to coastline"] / 1000

<xarray.DataArray (size: 5, application: 2, year: 1, value: 1)> Size: 80B
array([[[[52.43715437]],

        [[52.43715437]]],


       [[[52.43715437]],

        [[52.43715437]]],


       [[[52.43715437]],

        [[52.43715437]]],


       [[[52.43715437]],

        [[52.43715437]]],


       [[[52.43715437]],

        [[52.43715437]]]])
Coordinates:
  * size         (size) int64 40B 100 500 1000 3000 8000
  * application  (application) <U8 64B 'offshore' 'onshore'
    parameter    <U36 144B 'distance to coastline'
  * year         (year) int64 8B 2020
  * value        (value) int64 8B 0